In [ ]:
%load_ext autoreload
%autoreload 2
%pylab inline

In [ ]:
import numpy as np
import tensorflow as tf
import os, sys, time
os.environ["CUDA_VISIBLE_DEVICES"]=""
import utils, optimization, metrics, plot, data, evaluation
import functools
from data import fmap
import plot

# Parameters

In [ ]:
ns = 32
Mpch = 350
shift = 3
c = 20000
res = 256
def forward(x):
    return fmap.stat_forward(x, shift=shift, c=c)
def backward(x):
    return fmap.stat_backward(x, shift=shift, c=c)
def non_lin(x):
    return tf.nn.relu(x)

# Data handling

Load the data

In [ ]:
dataset = data.load.load_dataset(spix=ns, resolution=res,Mpch=Mpch, forward_map=forward)

# A) Load the model

In [ ]:
rootpath = '/scratch/snx3000/nperraud/saved_result/'
pathgan = rootpath+'WGAN32_stat_c_20000_shift_3_laplacian_Mpch_350_res_256_checkpoints/'
checkpoint = 45000

obj = evaluation.load_gan(pathgan=pathgan)



# Generate some samples

In [ ]:
N = 5000
gen_sample, gen_sample_raw = evaluation.generate_samples(obj, N=N, checkpoint=checkpoint)



Print a few samples

In [ ]:
plt.figure(figsize=(10,10))
plot.draw_images(gen_sample,nx=4,ny=4)
_ = plt.title('Generated')

In [ ]:
plt.figure(figsize=(10,10))
plot.draw_images(dataset.get_samples(N=16),nx=4,ny=4)
_ = plt.title('Real')

# Evaluate the model

In [ ]:
raw_images = backward(dataset.get_all_data())

### a) PSD

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_psd(raw_images, gen_sample_raw)


### b) Peak count

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_peak_cout(raw_images, gen_sample_raw)

### c) Mass histogram

In [ ]:
logel2, l2, logel1, l1 = evaluation.compute_and_plot_mass_hist(raw_images, gen_sample_raw)